In [ ]:
import spacy
import random
import numpy as np
import datasets
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn
import torch.nn.functional as F
import torch.optim as optim
from seq2seq import Decoder_GRU, Encoder_GRU, Seq2Seq, Seq2Seq_LSTM, Seq2Seq_GRU, Seq2Seq_GRU_Attention
from seq2seq import DOT_PRODUCT, SCALED_DOT_PRODUCT, GENERALIZED_DOT_PRODUCT
from datetime import datetime
from collections import Counter
from torch.utils.data import DataLoader
from torcheval.metrics.functional import bleu_score

In [2]:
device = torch.device('cpu')
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')

print(f"Run on device: {device}")

Run on device: mps


In [3]:
# ML cloud services
google_colab = False
azure_ml = False
kaggle = False

In [4]:
if google_colab:
    !mkdir -p data/Multi30k_HuggingFace
    !python -m spacy download en_core_web_sm
    !python -m spacy download de_core_news_sm
elif kaggle:
    !pip install torcheval
    !python -m spacy download en_core_web_sm
    !python -m spacy download de_core_news_sm
elif azure_ml:
    !python -m spacy download en_core_web_sm
    !python -m spacy download de_core_news_sm

In [5]:
dataset_path = "data/Multi30k_HuggingFace"
if azure_ml:
    dataset_path = "Users/khoi.hoangdai/" + "data/Multi30k_HuggingFace"
elif kaggle:
    dataset_path = "/kaggle/input/untitled"

In [6]:
dataset = datasets.load_dataset(dataset_path)
train_set, val_set, test_set = dataset['train'], dataset['validation'], dataset['test']
train_set[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [7]:
# Set random seeds
def setseed(seed):
    """Set all seeds and deterministic CuDNN behavior"""
    # Python random module
    random.seed(seed)
    
    # NumPy
    np.random.seed(seed)
    
    # PyTorch (CPU and all GPUs)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    # CuDNN configurations (critical for reproducibility)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

setseed(1711)

In [8]:
# Use tokenizer from spacy
en_nlp = spacy.load('en_core_web_sm')
de_nlp = spacy.load('de_core_news_sm')

In [9]:
# Build the token frequency dict, ignore tokens with low frequency
en_token_dict = Counter()
de_token_dict = Counter()
unk, pad, sos, eos = '<unk>', '<pad>', '<sos>', '<eos>'
special_tokens = [unk, pad, sos, eos]
min_freq = 2

for example in train_set:
    en_tokens = [token.text.lower() for token in en_nlp.tokenizer(example['en'])]
    de_tokens = [token.text.lower() for token in de_nlp.tokenizer(example['de'])]
    en_token_dict.update(en_tokens)
    de_token_dict.update(de_tokens)

# No need to keep track of the frequency
en_token_dict = [k for (k, v) in en_token_dict.items() if v >= min_freq]
en_token_dict = special_tokens + en_token_dict
en_token_dict = {value: index for (index, value) in enumerate(en_token_dict)}
en_idx_token_dict = {value: key for (key, value) in en_token_dict.items()}

de_token_dict = [k for (k, v) in de_token_dict.items() if v >= min_freq]
de_token_dict = special_tokens + de_token_dict
de_token_dict = {value: index for (index, value) in enumerate(de_token_dict)}
de_idx_token_dict = {value: key for (key, value) in de_token_dict.items()}

In [10]:
# Check if special tokens share the same index
for special in special_tokens:
    if not en_token_dict[special] == de_token_dict[special]:
        print(f"Token {special} mismatch between EN and DE dictionary")

In [11]:
# Create token list and token IDs for each sentence in the dataset
def tokenize_example(example, en_nlp, de_nlp, en_token_dict, de_token_dict, sos, eos):
    en_tokens, de_tokens = [], []
    en_ids, de_ids = [], []
    for token in en_nlp.tokenizer(example['en']):
        token = token.text.lower()
        if token not in en_token_dict:
            token = unk

        en_tokens.append(token)
        en_ids.append(en_token_dict[token])

    en_tokens = [sos] + en_tokens + [eos]
    en_ids = [en_token_dict[sos]] + en_ids + [en_token_dict[eos]]

    for token in de_nlp.tokenizer(example['de']):
        token = token.text.lower()
        if token not in de_token_dict:
            token = unk

        de_tokens.append(token)
        de_ids.append(de_token_dict[token])

    de_tokens = [sos] + de_tokens + [eos]
    de_ids = [de_token_dict[sos]] + de_ids + [de_token_dict[eos]]

    example['en_tokens'] = en_tokens
    example['en_ids'] = en_ids
    example['de_tokens'] = de_tokens
    example['de_ids'] = de_ids

    return example


In [12]:
fn_kwargs = {
    'en_nlp': en_nlp,
    'de_nlp': de_nlp,
    'en_token_dict': en_token_dict,
    'de_token_dict': de_token_dict,
    'sos': sos,
    'eos': eos,
}
train_set = train_set.map(tokenize_example, fn_kwargs=fn_kwargs)
val_set = val_set.map(tokenize_example, fn_kwargs=fn_kwargs)
test_set = test_set.map(tokenize_example, fn_kwargs=fn_kwargs)

In [13]:
print(train_set[0]['de'])
print(train_set[0]['de_tokens'])
print(train_set[0]['de_ids'])
print(train_set[0]['en'])
print(train_set[0]['en_tokens'])
print(train_set[0]['en_ids'])

Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
['<sos>', 'zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.', '<eos>']
[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 3]
Two young, White males are outside near many bushes.
['<sos>', 'two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.', '<eos>']
[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 3]


In [14]:
# Write a collate_fn to pad sequences with variable length into a batch of tensors for Dataloader
def get_collate_fn(pad_index=1):
    def collate_fn(batch):
        # Encoder input: <sequence> + <eos>
        encoder_input = [torch.tensor(sequence['de_ids'][1:]) for sequence in batch]
        encoder_input = rnn.pad_sequence(encoder_input, padding_value=pad_index, batch_first=True)

        # Decode input: <sos> + <sequence>
        decoder_input = [torch.tensor(sequence['en_ids'][:-1]) for sequence in batch]
        decoder_input = rnn.pad_sequence(decoder_input, padding_value=pad_index, batch_first=True)

        # Decode output: <sequence> + <eos>
        decoder_output = [torch.tensor(sequence['en_ids'][1:]) for sequence in batch]
        decoder_output = rnn.pad_sequence(decoder_output, padding_value=pad_index, batch_first=True)

        return encoder_input, decoder_input, decoder_output

    return collate_fn

In [15]:
collate_fn = get_collate_fn()
batch_size = 128
train_dl = DataLoader(train_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)

In [16]:
def train_fn(model, optimizer, epochs, loss_fn=F.cross_entropy, dataloader=train_dl, pad_idx=1):
    total_loss = 0
    for epoch in range(epochs):
        epoch_loss = 0
        epoch_start = datetime.now()
        next_chunk = 0
        for idx, dl in enumerate(dataloader):
            batch_start = datetime.now()
            encoder_input, decoder_input, decoder_output = dl
            encoder_input = encoder_input.to(device)
            decoder_input = decoder_input.to(device)
            decoder_output = decoder_output.to(device)
            output = model(encoder_input, decoder_input)[0]
            loss = loss_fn(output.permute(0, 2, 1), decoder_output, ignore_index=pad_idx, reduction='mean')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            batch_runtime = datetime.now() - batch_start
            if idx == next_chunk:
                print(f"Chunk={idx}: loss={loss.item():.2f}, batch runtime={batch_runtime.total_seconds()*1000:.2f} ms")
                next_chunk += len(train_dl) // 10

        total_loss += epoch_loss
        epoch_runtime = datetime.now() - epoch_start
        print(f"Epoch={epoch}: Loss={epoch_loss / len(train_dl):.2f}, epoch runtime={epoch_runtime.seconds:.2f} seconds")

    return total_loss / len(train_dl)

In [ ]:
input_dim = len(de_token_dict)
output_dim = len(en_token_dict)
input_emb_dim = 256
output_emb_dim = 256
rnn_hidden_dim = 512
rnn_num_layers = 4
bidirectional = True
epochs = 20

# Seq2Seq_GRU_DotProductAttention
attention_type = DOT_PRODUCT
seq2seq_gru_dotprod_attn_model = Seq2Seq_GRU_Attention(input_dim, input_emb_dim,
                                                 output_dim, output_emb_dim,
                                                 rnn_hidden_dim, rnn_num_layers, attention_type,
                                                 bidirectional=bidirectional).to(device)
seq2seq_gru_dotprod_attn_optimizer = optim.Adam(seq2seq_gru_dotprod_attn_model.parameters(), lr=1e-3)


In [ ]:
MODEL_DIR = "saved_weights"
MODEL_FILE = f"seq2seq_gru_attn={attention_type}_bidirectional={bidirectional}_layer={rnn_num_layers}_epochs={epochs}.pt"

In [ ]:
model = seq2seq_gru_dotprod_attn_model
optimizer = seq2seq_gru_dotprod_attn_optimizer
train_gru_err = train_fn(model, optimizer, epochs)
torch.save(model.state_dict(), MODEL_FILE)

In [18]:
def get_blue_tokenizer(en_nlp, en_token_dict, unk):
    def blue_tokenizer(s):
        en_tokens = [token.text.lower() if token.text.lower() in en_token_dict else unk for token in en_nlp.tokenizer(s)]
        return en_tokens

    return blue_tokenizer

In [19]:
def en_idx_to_sentence(indices, en_idx_token_dict, pad_idx=1):
    sentence = [en_idx_token_dict[idx.item()] for idx in indices if idx.item() != pad_idx]
    return " ".join(sentence)

In [20]:
def evaluate(model, model_file, val_set=val_set, collate_fn=collate_fn, batch_size=batch_size, en_idx_token_dict=en_idx_token_dict, eos=eos, device=device, max_output_len=30):
    val_dl = DataLoader(val_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
    # tokenizer_fn = get_blue_tokenizer(en_nlp, en_token_dict, unk)
    all_bleu_scores = []
    with open(f"bleu_output/{model_file}_bleu.txt", "w") as f:
        with torch.no_grad():
            for idx, dl in enumerate(val_dl):
                encoder_input, _, decoder_output = dl
                encoder_input = encoder_input.to(device)
                decoder_output = decoder_output.to(device)

                for i, seq_input in enumerate(encoder_input):
                    translated = model.translate(seq_input, en_idx_token_dict=en_idx_token_dict, device=device, eos=eos, sos_idx=2, max_output_len=max_output_len)
                    #  = model.translate(model, seq_input, en_idx_token_dict, device)
                    en_groud_truth = en_idx_to_sentence(decoder_output[i], en_idx_token_dict)
                    translated_sentence = " ".join(translated)
                    
                    try:
                        blue_results = bleu_score([" ".join(translated)], [en_groud_truth])
                        print(en_groud_truth)
                        f.write(en_groud_truth + "\n")
                        print(translated_sentence)
                        f.write(translated_sentence + "\n")
                        print(blue_results)        
                        f.write(str(blue_results.item()) + "\n")
                        f.write("=======================" +  "\n")

                        all_bleu_scores.append(blue_results.item())
                    except ValueError as e:
                        print(f"An unexpected error occurred: {e}")

        avg_bleu_score = np.mean(all_bleu_scores)
        f.write(str(avg_bleu_score) + "\n")
        print(avg_bleu_score)

In [ ]:
MODEL_DIR = "saved_weights"

for bidirectional in [False]:
    for attention_type in S
        MODEL_FILE = f"seq2seq_gru_attn={attention_type}_bidirectional={bidirectional}_layer={rnn_num_layers}_epochs={epochs}.pt"
        # Load the Seq2Seq model by first initializing the architecture of Encoder and Decoder
        reload_seq2seq = Seq2Seq_GRU_Attention(input_dim, input_emb_dim,
                                                 output_dim, output_emb_dim,
                                                 rnn_hidden_dim, rnn_num_layers, attention_type,
                                                 bidirectional=bidirectional).to(device)
        reload_seq2seq.load_state_dict(torch.load(MODEL_DIR + "/" + MODEL_FILE, weights_only=False, map_location=device))
        evaluate(reload_seq2seq, MODEL_FILE)

a competitive runner taking her first <unk> in a competition . <eos>
a female runner <unk> for a great free festival . <eos>
tensor(0.)
multiple bodies collide in a soccer match . <eos>
several people are posing for a picture in a field . <eos>
tensor(0.)
many people in a stadium dressed in white are conversing with each other . <eos>
a large group of people in yellow and yellow clothing navigates up into a body of water . <eos>
tensor(0.)
this is some sort of band or symphony practicing . <eos>
this is our night for a wedding , but all at her heart . <eos>
tensor(0.)
a young boy laying on a hospital bed with his leg over the side . <eos>
a young boy is lying on a toy tunnel dressed in green and yellow . <eos>
tensor(0.)
a boy in a red shirt trying to play a guitar . <eos>
a boy in a red shirt plays the trumpet . <eos>
tensor(0.4330)
a little girl peering over a blue wall . <eos>
a little girl is sliding down a snowbank at a playground . <eos>
tensor(0.)
a boy band and no one even matc